In [1]:
import pandas as pd
import numpy as np
import itertools

# Importing Data



In [2]:
#example source data

evidence = pd.read_csv('Source Data For Reference/Evidence1020.csv')
language = pd.read_csv('Source Data For Reference/Language1020.csv')
probability = pd.read_csv('Source Data For Reference/Probability1020.csv')
reasoning = pd.read_csv('Source Data For Reference/Reasoning1020.csv')

In [3]:
#OLD FAKE GOLD STANDARD DATA - IGNORE
gold_reasoning = pd.read_csv('Gold Standard Psuedo Data/GoldStandardReferenceReasoning.csv')
gold_evidence = pd.read_csv('Gold Standard Psuedo Data/GoldStandardReferenceEvidence.csv')
gold_language = pd.read_csv('Gold Standard Psuedo Data/GoldStandardReferenceLanguage.csv')
gold_probability = pd.read_csv('Gold Standard Psuedo Data/GoldStandardReferenceProbability.csv')

In [4]:
#OLD IAA DATA - IGNORE
iaa = pd.read_csv('iaa.csv')


FileNotFoundError: [Errno 2] File b'iaa.csv' does not exist: b'iaa.csv'

In [5]:
# Loading schema and iaa_new_format
schema = pd.read_csv('Covid_Evidence2020_03_21-Schema.csv')
iaa_new_format = pd.read_csv('Covid_Evidencev1.IAA-2020-11-17T0612-Tags.csv')

In [6]:
#Testing out merging with schema

answers = iaa_new_format.merge(schema, how="left", on="answer_uuid")
answers = answers[["article_number", "article_sha256", "schema_sha256", "answer_uuid", "source_task_uuid",
                     "target_text", "question_uuid", "question_label", "answer_label",
                     "question_type", "answer_count"]]

answers = answers.replace(np.nan, '', regex=True)


gold_standard_new_data = pd.read_csv('Covid_Evidencev1.adjudicated-2021-01-11T0253-Tags.csv')
gold_answers = gold_standard_answers.merge(schema, how='left', on = "answer_uuid")


NameError: name 'gold_standard_answers' is not defined

# Functions

In [16]:
#USE THIS FUNCTION, takes in source data, gold data, schema, and new iaa format data and adds the gold standard and iaa column

def together_with_new_formatv2(data_hunt, gold_data, schema, iaa_data):
    
    #merge the new iaa data with the schema 
    iaa_selected = iaa_data.merge(schema, how="left", on="answer_uuid")
    iaa_selected = iaa_selected[["article_number", "article_sha256", "schema_sha256", "answer_uuid", "source_task_uuid",
                     "target_text", "question_uuid", "question_label", "answer_label",
                     "question_type", "answer_count"]]
    iaa_selected = iaa_selected.replace(np.nan, '', regex=True)
    
   
    #merges gold standard adjudidcated data with schema 
    gold_answers = gold_data.merge(schema, how='left', on = "answer_uuid")
    #renaming answer_label to gold_standard label to make it more readable
    gold_answers.rename(columns={"answer_label": "Gold_Standard_Label"}, inplace=True)
    gold_answers_final = gold_answers[['article_number', 'question_label', 'Gold_Standard_Label', 'answer_uuid']]
    #THIS IS WHERE I DROPPED ROWS ASSUMING ONE ANSWER PER QUESTION FOR GOLD ANSWER. 
    #Looks like the adjudicated data includes an entry for different highlights and such as well. So dropped duplicated 
    gold_answers_final = gold_answers_final.drop_duplicates(['article_number','question_label'], keep= 'last')
    #merged with the source data (ex. evidence)
    data_with_gold = pd.merge(data_hunt, gold_answers_final, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)

    #fix up the now merged iaa data
    iaa_selected = iaa_selected[['article_number', 'question_label', 'answer_label']]
    #renaming answer_label to iaa_label_with_duplicate to make it more readable
    iaa_selected = iaa_selected.rename(columns={"answer_label": "iaa_label_with_duplicate"})
    iaa_final = iaa_selected.groupby(['article_number', 'question_label'])['iaa_label_with_duplicate'].apply(list).reset_index()
    
    #put the gold standard data + evidence with the merged iaa data
    data_with_gold_and_iaa = pd.merge(data_with_gold, iaa_final, how = 'left',  on = ['article_number', 'question_label'])
    data_with_gold_and_iaa.fillna('', inplace=True)
    data_with_gold_and_iaa['iaa_label'] = data_with_gold_and_iaa['iaa_label_with_duplicate'].apply(lambda x: list(set(x)))
    del data_with_gold_and_iaa['iaa_label_with_duplicate']
    del data_with_gold_and_iaa['answer_uuid_y']
    data_with_gold_and_iaa.to_csv('evidence_with_gold_and_iaa_new_formatv2.csv')
    return data_with_gold_and_iaa

In [17]:
together_with_new_formatv2(evidence, gold_standard_new_data, schema, iaa_new_format)

,namespace,schema_sha256,quiz_task_uuid,task_url,tua_uuid,article_batch_name,article_number,article_filename,article_sha256,article_text_length,...,answer_uuid_x,submitted_tua_uuid,answer_text,case_number,highlight_count,start_pos,end_pos,target_text,Gold_Standard_Label,iaa_label
0,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,b5c3c5d0-4c59-4e26-8996-97cd64e29b93,01b6f358-3214-4bbc-9408-40d0d0cac421,No causal claim made,0,0,0,0,,T1.Q1.A1,"[T1.Q1.A3, T1.Q1.A1]"
1,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,27ebbcb0-94fa-4477-8644-af3f6c9ea03b,e614d272-db0c-4450-b9de-9342b75288b8,No,0,0,0,0,,T1.Q9.A1,"[T1.Q9.A2, T1.Q9.A1]"
2,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,e2e6a7d3-ad12-4ba6-ab81-f7f73bc68a83,f6f6677d-b963-42ba-a735-ee8e4a9cb98b,"No, not at all",0,0,0,0,,T1.Q12.A2,"[T1.Q12.A4, T1.Q12.A2, T1.Q12.A3]"
3,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,73d8f381-e2b9-41fd-ac96-e17752b6c26a,eabc18cd-a9da-408b-a5d4-be592061164d,5: Middling difficulty,0,0,0,0,,T1.Q13.A5,"[T1.Q13.A5, T1.Q13.A4]"
4,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,d4696215-f81b-46da-b965-b9377f68e6bb,7407b08d-a6fd-4b20-9d8e-b61db7751228,5: 50-50,0,0,0,0,,T1.Q14.A5,"[T1.Q14.A5, T1.Q14.A7]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,a0972088-cb65-4764-a4ee-3be2b2bd100a,3da3704f-ff60-4a0a-942d-28037e9c4ff3,"Yes, implicitly",1,3,3030,3035,might,T1.Q12.A2,"[T1.Q12.A4, T1.Q12.A2, T1.Q12.A3]"
70,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,a0972088-cb65-4764-a4ee-3be2b2bd100a,3da3704f-ff60-4a0a-942d-28037e9c4ff3,"Yes, implicitly",1,3,2559,2584,can reduce the likelihood,T1.Q12.A2,"[T1.Q12.A4, T1.Q12.A2, T1.Q12.A3]"
71,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,24657063-8eb5-4cf4-83d1-f39945922c3f,https://pe.goodlylabs.org/project/Covid_Eviden...,6def4416-bb6d-4d6c-a9a3-e8cd0b9891e7,CovidArticles/Covid_ar

# Old Functions

In [179]:
#OLD gold standard FUNCTIONS - NOT USED ANYMORE

def create_gold_standard_column_language(data_hunt, gold_data):
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)
    data_with_gold.to_csv('language_with_gold.csv')
    return data_with_gold
   
def create_gold_standard_column_reasoning(data_hunt, gold_data):
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)
    data_with_gold.to_csv('reasoning_with_gold.csv')
    return data_with_gold
   
def create_gold_standard_column_probability(data_hunt, gold_data):
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)
    data_with_gold.to_csv('probability_with_gold.csv')
    return data_with_gold
   
def create_gold_standard_column_evidence(data_hunt, gold_data):
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)
    data_with_gold.to_csv('evidence_with_gold.csv')
    return data_with_gold
   
    
#OLD - this didn't use the new IAA format. Ignore!

def together(data_hunt, gold_data, schema, iaa_data):
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)

  
    iaa_selected = iaa_data[['article_num', 'question_Number', 'agreed_Answer']]
    iaa_selected = iaa_selected.rename(columns={"article_num": "article_number"})
    iaa_selected = iaa_selected[~iaa_selected.agreed_Answer.str.contains("L")]
    iaa_selected['iaa_label'] = "T1.Q" + iaa_selected['question_Number'].astype(str) + ".A" + iaa_selected['agreed_Answer'].astype(str)
    iaa_selected['question_label'] = "T1.Q" + iaa_selected['question_Number'].astype(str)
    iaa_final = iaa_selected.groupby(['article_number', 'question_label'])['iaa_label'].apply(list).reset_index()


    data_with_gold_and_iaa = pd.merge(data_with_gold, iaa_final, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold_and_iaa.fillna('', inplace=True)
    data_with_gold_and_iaa.to_csv('evidence_with_gold_and_iaa.csv')
    return data_with_gold_and_iaa

#OLD - this didn't use the gold standard adjudicated data. Ignore!

def together_with_new_format(data_hunt, gold_data, schema, iaa_data):
    
    iaa_selected = iaa_data.merge(schema, how="left", on="answer_uuid")
    iaa_selected = iaa_selected[["article_number", "article_sha256", "schema_sha256", "answer_uuid", "source_task_uuid",
                     "target_text", "question_uuid", "question_label", "answer_label",
                     "question_type", "answer_count"]]

    iaa_selected = iaa_selected.replace(np.nan, '', regex=True)
    data_with_gold = pd.merge(data_hunt, gold_data, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold.fillna('', inplace=True)

  
    iaa_selected = iaa_selected[['article_number', 'question_label', 'answer_label']]
    iaa_selected = iaa_selected.rename(columns={"answer_label": "iaa_label_with_duplicate"})
    iaa_final = iaa_selected.groupby(['article_number', 'question_label'])['iaa_label_with_duplicate'].apply(list).reset_index()
    

    data_with_gold_and_iaa = pd.merge(data_with_gold, iaa_final, how = 'left', on = ['article_number', 'question_label'])
    data_with_gold_and_iaa.fillna('', inplace=True)
    data_with_gold_and_iaa['iaa_label'] = data_with_gold_and_iaa['iaa_label_with_duplicate'].apply(lambda x: list(set(x)))
    del data_with_gold_and_iaa['iaa_label_with_duplicate']
    data_with_gold_and_iaa.to_csv('evidence_with_gold_and_iaa_new_format.csv')
    return data_with_gold_and_iaa
    
    
    
    
    
    
    
    
    
    
    
    